### I. Scrape financial data from Yahoo Finance

In [1]:
# import libraries
import yfinance as yf
import pandas as pd
import datetime

In [2]:
# Download all stock price historical data from yahoo finance
spy_data = yf.download("SPY")
aapl_data = yf.download("AAPL")
goog_data = yf.download("GOOG")
tsla_data = yf.download("TSLA")
amzn_data = yf.download("AMZN")
meta_data = yf.download("META")
nvda_data = yf.download("NVDA")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
# Reset index to turn date to a separate column
spy_data.reset_index(inplace=True)
aapl_data.reset_index(inplace=True)
goog_data.reset_index(inplace=True)
tsla_data.reset_index(inplace=True)
amzn_data.reset_index(inplace=True)
meta_data.reset_index(inplace=True)
nvda_data.reset_index(inplace=True)

In [4]:
# Add a column for ticker symbol
spy_data['Ticker']  = 'SPY'
aapl_data['Ticker'] = 'AAPL'
goog_data['Ticker'] = 'GOOG'
tsla_data['Ticker'] = 'TSLA'
amzn_data['Ticker'] = 'AMZN'
meta_data['Ticker'] = 'META'
nvda_data['Ticker'] = 'NVDA'

In [5]:
# Preview dataframe
spy_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,1993-01-29,43.96875,43.96875,43.75000,43.93750,24.840672,1003200,SPY
1,1993-02-01,43.96875,44.25000,43.96875,44.25000,25.017361,480500,SPY
2,1993-02-02,44.21875,44.37500,44.12500,44.34375,25.070360,201300,SPY
3,1993-02-03,44.40625,44.84375,44.37500,44.81250,25.335360,529400,SPY
4,1993-02-04,44.96875,45.09375,44.46875,45.00000,25.441378,531500,SPY


In [6]:
# Put all stocks into a consolidated data frame
stock_all = pd.concat([spy_data, aapl_data, goog_data, tsla_data, amzn_data, meta_data, nvda_data])

In [7]:
# Restrict data to work with to only dates after specific year
stock_all['Date'] = pd.to_datetime(stock_all['Date'])
df = stock_all[stock_all['Date'] >= '2010-01-01']

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24024 entries, 4264 to 6288
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       24024 non-null  datetime64[ns]
 1   Open       24024 non-null  float64       
 2   High       24024 non-null  float64       
 3   Low        24024 non-null  float64       
 4   Close      24024 non-null  float64       
 5   Adj Close  24024 non-null  float64       
 6   Volume     24024 non-null  int64         
 7   Ticker     24024 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.6+ MB


### II. DATA VISUALIZATION USING BOKEH

In [9]:
from bokeh.layouts import layout, column, row
from bokeh.models import ColumnDataSource, RangeSlider, DateRangeSlider, HoverTool, CDSView, BooleanFilter, CustomJS, Dropdown, Select, Div
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [10]:
# Define date range for graph
start_date = pd.to_datetime('2010-01-01')
end_date   = datetime.datetime.now()
date_rangeX = pd.date_range(start = start_date, end = end_date)

# Create lists for ticker & color
ticker_list = ['SPY',       'AAPL',    'AMZN',     'GOOG',   'META',       'NVDA',  'TSLA']
color_list  = ['lightblue', 'red',   'orange', 'darkblue',  'olive',  'darkgreen',  'pink']

# Instantiating the figure object
graph = figure(title = f"Stock Closing Prices {start_date.year}-{end_date.year}",
               x_axis_type = "datetime",
               x_axis_label = 'Date', 
               y_axis_label = 'Price (USD)',
               sizing_mode = "stretch_width",
               height = 500)

# Make font size of title bigger
graph.title.text_font_size = '20pt'
graph.title.align = "center"


# Draw graphs for all ticker on the list
for data, name, color in zip([df[df['Ticker']== 'SPY'],    df[df['Ticker']== 'AAPL'], 
                              df[df['Ticker']== 'AMZN'],   df[df['Ticker']== 'GOOG'], 
                              df[df['Ticker']== 'META'],   df[df['Ticker']== 'NVDA'], 
                              df[df['Ticker']== 'TSLA']], 
                              ticker_list, 
                              color_list ):
    graph.line(data['Date'], data['Adj Close'], line_width=1.5, color=color, alpha=0.8, legend_label=name)
    
# Put legend to top left corner & define click_policy
graph.legend.location = "top_left"
graph.legend.click_policy = "hide"

# Set up RangeSlider for timeline
timeline_slider = DateRangeSlider(    title = "Adjust timeline (x-axis) range",
                                      start = date_rangeX.min(),
                                      end   = date_rangeX.max(),
                                      step  = 1,
                                      value = (date_rangeX.min(), date_rangeX.max()),
                                      sizing_mode = "stretch_width")

# Add callback code
timeline_slider.js_link("value", graph.x_range, "start", attr_selector=0)
timeline_slider.js_link("value", graph.x_range, "end", attr_selector=1)

# Define the tooltip
hover_tool = HoverTool(  tooltips   = [   ('Date', '@x{%F}'),  ('Adj Price',  '@y{$0.00}' )   ],
                         formatters = {'@x': 'datetime'}  )

# Add hover_tool to graph
graph.add_tools(hover_tool)

# create layout
layout = column(graph, timeline_slider, sizing_mode = 'stretch_both')

# show result
show(layout)